## Databases 

In [15]:
import os
import pandas as pd
import time

In [3]:
hostname = "127.0.0.1"
dataDir = os.listdir("../data/full/")
dataDir

['.ipynb_checkpoints',
 'hospital-testData.csv',
 'hospital-testData.json',
 'hospital-tab.csv',
 'hospital-tab.json',
 'college-tab.json']

In [4]:
hospital = pd.read_csv("../data/full/hospital-tab.csv").to_dict("records")

In [5]:
college = pd.read_json("../data/full/college-tab.json").to_dict("records")

### Mongo 

In [71]:
from pymongo import MongoClient

In [344]:
client = MongoClient(host=hostname,port=27017)

In [345]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
[[e.update(id=idx+1) for idx, e in enumerate(data)] for data in [hospital, college]]

In [ ]:
hospitalMongo = client["hospital"]["hospital"]
collegeMongo = client["college"]["college"]

In [ ]:
hospitalMongo.insert_many(hospital)
collegeMongo.insert_many(college)

### Neo4j 

In [6]:
hospital[0].keys()
college[0].keys()

dict_keys(['Name', 'Address', 'City', 'Zip', 'Web', 'ProgramName'])

In [347]:
#from neo4j import GraphDatabase

In [7]:
from py2neo import Graph, Node, Relationship

In [8]:
txpool = []  
for e in hospital: 
    start = Node("Hospital",name=e["start_name"],lat=e["start_loc_lat"],lon=e["start_loc_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_loc_lat"],lon=e["end_loc_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
        
    txpool.append(start)
    txpool.append(end)
    txpool.append(r)

In [10]:
unique = set([e["Name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["Name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["ProgramName"]), pred))
                              
    university = Node("University", name=pred[0]["Name"], web=pred[0]["Web"], city=pred[0]["City"])    
    [txpool.append(Relationship(university,"TEACHES",e)) for e in getProgramName]
    
    print(f"{idx} - {collegeName} - {len(pred)} - {len(getProgramName)}")

[DEBUG::] 0 - Elizabethtown Community and Technical College - 2 - 2
[DEBUG::] 1 - Suffolk University - 6 - 6
[DEBUG::] 2 - Baystate Medical Center - 2 - 2
[DEBUG::] 3 - Brightwood Career Institute - ICM Campus - 1 - 1
[DEBUG::] 4 - American Career College - 4 - 4
[DEBUG::] 5 - Azusa Pacific University - 12 - 12
[DEBUG::] 6 - Susquehanna University - 2 - 2
[DEBUG::] 7 - University of Wisconsin - Green Bay - 5 - 5
[DEBUG::] 8 - Delta State University - 6 - 6
[DEBUG::] 9 - Chadron State College - 1 - 1
[DEBUG::] 10 - Odessa College - 4 - 4
[DEBUG::] 11 - Baptist Health System - 2 - 2
[DEBUG::] 12 - Bainbridge State College - 1 - 1
[DEBUG::] 13 - Stockton University - 8 - 8
[DEBUG::] 14 - Dordt College - 1 - 1
[DEBUG::] 15 - Lynn University - 1 - 1
[DEBUG::] 16 - Florida State College at Jacksonville - 9 - 9
[DEBUG::] 17 - Bridgerland Applied Technology College - 2 - 2
[DEBUG::] 18 - American Institute of Alternative Medicine - 2 - 2
[DEBUG::] 19 - Pasadena Conservatory of Music - 1 - 1
[D

[DEBUG::] 180 - Jefferson Community College - 2 - 2
[DEBUG::] 181 - Georgetown College - 1 - 1
[DEBUG::] 182 - Rend Lake College - 2 - 2
[DEBUG::] 183 - Metropolitan Community College - Kansas City - 11 - 11
[DEBUG::] 184 - Saint Louis Community College - 17 - 17
[DEBUG::] 185 - Community Hospital - 1 - 1
[DEBUG::] 186 - Maricopa Community Colleges - Glendale Community College - 1 - 1
[DEBUG::] 187 - Fielding Graduate University - 1 - 1
[DEBUG::] 188 - Guilford Technical Community College - 4 - 4
[DEBUG::] 189 - Nichols Career Center - 3 - 3
[DEBUG::] 190 - Temple College - 2 - 2
[DEBUG::] 191 - Atascadero State Hospital - 1 - 1
[DEBUG::] 192 - Good Samaritan College of Nursing and Health Science - 3 - 3
[DEBUG::] 193 - Brightwood College - North Hollywood - 1 - 1
[DEBUG::] 194 - Central Texas College - 1 - 1
[DEBUG::] 195 - Drury University - 2 - 2
[DEBUG::] 196 - Rush University - 14 - 14
[DEBUG::] 197 - The New School - 1 - 1
[DEBUG::] 198 - Conservatory of Music of Puerto Rico - 1 

[DEBUG::] 367 - Northern New Mexico College - 3 - 3
[DEBUG::] 368 - Music Institute of Chicago - 1 - 1
[DEBUG::] 369 - Bakersfield College - 1 - 1
[DEBUG::] 370 - VA New Jersey Health Care System - East Orange - 1 - 1
[DEBUG::] 371 - Pasadena City College - 5 - 5
[DEBUG::] 372 - Columbus State Community College - 4 - 4
[DEBUG::] 373 - University of Massachusetts Amherst - 14 - 14
[DEBUG::] 374 - Long Beach City College - 2 - 2
[DEBUG::] 375 - Southwestern College - 6 - 6
[DEBUG::] 376 - Northeastern Illinois University - 3 - 3
[DEBUG::] 377 - Center for Instruction, Technology, and Innovation (CiTi) - 1 - 1
[DEBUG::] 378 - The University of Texas at El Paso - 9 - 9
[DEBUG::] 379 - South Arkansas Community College - 3 - 3
[DEBUG::] 380 - Applewood Centers - 1 - 1
[DEBUG::] 381 - California State Polytechnic University, Pomona - 5 - 5
[DEBUG::] 382 - University of Louisiana at Monroe - 12 - 12
[DEBUG::] 383 - Concorde Career Institute - 6 - 6
[DEBUG::] 384 - SUNY Potsdam - 3 - 3
[DEBUG::

[DEBUG::] 549 - West Virginia University at Parkersburg - 2 - 2
[DEBUG::] 550 - San Juan College - 4 - 4
[DEBUG::] 551 - Moraine Park Technical College - 3 - 3
[DEBUG::] 552 - Gateway Seminary - 1 - 1
[DEBUG::] 553 - Central Georgia Technical College - 4 - 4
[DEBUG::] 554 - East Mississippi Community College - 2 - 2
[DEBUG::] 555 - Averett University - 1 - 1
[DEBUG::] 556 - University of Minnesota - Duluth - 4 - 4
[DEBUG::] 557 - SUNY College of Agriculture & Technology at Morrisville - 4 - 4
[DEBUG::] 558 - Clinton Technical School - 1 - 1
[DEBUG::] 559 - Texas Health and Science University - 2 - 2
[DEBUG::] 560 - Northeast Ohio Medical University - 3 - 3
[DEBUG::] 561 - University of South Carolina Upstate - 3 - 3
[DEBUG::] 562 - Nash Community College - 1 - 1
[DEBUG::] 563 - Schuylkill Medical Center - S. Jackson Street - 1 - 1
[DEBUG::] 564 - New York Language Center New York Language Center / Midtown - 3 - 3
[DEBUG::] 565 - MedCentral College of Nursing - 1 - 1
[DEBUG::] 566 - Sea

[DEBUG::] 732 - Ulster County Community College - 2 - 2
[DEBUG::] 733 - Bethune - Cookman University - 2 - 2
[DEBUG::] 734 - Indiana University - Purdue University Indianapolis - 35 - 35
[DEBUG::] 735 - Belmont Abbey College - 1 - 1
[DEBUG::] 736 - Mercy Medical Center School of Radiology - 1 - 1
[DEBUG::] 737 - Northern Oklahoma College - 1 - 1
[DEBUG::] 738 - Taft College - 1 - 1
[DEBUG::] 739 - Wiregrass Georgia Technical College - 4 - 4
[DEBUG::] 740 - Ozarks Technical Community College - 8 - 8
[DEBUG::] 741 - Siena College - 2 - 2
[DEBUG::] 742 - George Washington University, The - 15 - 15
[DEBUG::] 743 - Walters State Community College - 2 - 2
[DEBUG::] 744 - American Academy of Acupuncture and Oriental Medicine - 2 - 2
[DEBUG::] 745 - Rochester Community and Technical College - 4 - 4
[DEBUG::] 746 - Olympic College - 3 - 3
[DEBUG::] 747 - University of La Verne - 2 - 2
[DEBUG::] 748 - Ingalls Memorial Hospital - 1 - 1
[DEBUG::] 749 - Globe Institute of Technology - 1 - 1
[DEBUG:

...

[DEBUG::] 2019 - Lakeland College - 2 - 2
[DEBUG::] 2020 - Richmond VA Medical Center - 3 - 3
[DEBUG::] 2021 - Whitworth University - 3 - 3
[DEBUG::] 2022 - Pennsylvania State University - 44 - 44
[DEBUG::] 2023 - Bon Secours St. Mary's Hospital - 2 - 2
[DEBUG::] 2024 - Arnot Ogden Medical Center - 2 - 2
[DEBUG::] 2025 - Pennsylvania College of Health Sciences - 3 - 3
[DEBUG::] 2026 - University of Northwestern - St. Paul - 2 - 2
[DEBUG::] 2027 - Blessing-Rieman College of Nursing - 3 - 3
[DEBUG::] 2028 - Augusta Technical College - 4 - 4
[DEBUG::] 2029 - Portland VA Medical Center - 6 - 6
[DEBUG::] 2030 - Northwestern University - 16 - 16
[DEBUG::] 2031 - Harrisburg Area Community College - 5 - 5
[DEBUG::] 2032 - Tompkins-Cortland Community College - 2 - 2
[DEBUG::] 2033 - Florida Gulf Coast University - 10 - 10
[DEBUG::] 2034 - The Diller-Quaile School of Music - 1 - 1
[DEBUG::] 2035 - University of Wisconsin - Superior - 2 - 2
[DEBUG::] 2036 - Quinsigamond Community College - 6 - 6


[DEBUG::] 2207 - American Academy of Dramatic Arts - 2 - 2
[DEBUG::] 2208 - San Francisco State University - 12 - 12
[DEBUG::] 2209 - Alderson Broaddus University - 3 - 3
[DEBUG::] 2210 - Yale University - 14 - 14
[DEBUG::] 2211 - Roseman University of Health Sciences - 5 - 5
[DEBUG::] 2212 - Polk State College - 5 - 5
[DEBUG::] 2213 - Bradley University - 12 - 12
[DEBUG::] 2214 - Florida International University - 26 - 26
[DEBUG::] 2215 - University of South Carolina - Beaufort - 2 - 2
[DEBUG::] 2216 - Shawnee State University - 9 - 9
[DEBUG::] 2217 - SUNY Downstate Medical Center - 12 - 12
[DEBUG::] 2218 - Coastal Carolina Community College - 2 - 2
[DEBUG::] 2219 - East Stroudsburg University of Pennsylvania - 4 - 4
[DEBUG::] 2220 - Anthem College - Maryland Heights - 2 - 2
[DEBUG::] 2221 - Rogers State University - 2 - 2
[DEBUG::] 2222 - Eastern Florida State College - 4 - 4
[DEBUG::] 2223 - Berkshire Community College - 2 - 2
[DEBUG::] 2224 - Southside Virginia Community College - 

[DEBUG::] 2390 - Harding University - 12 - 12
[DEBUG::] 2391 - Western Nebraska Community College - 1 - 1
[DEBUG::] 2392 - Stevenson University - 5 - 5
[DEBUG::] 2393 - Emory University - 18 - 18
[DEBUG::] 2394 - Washtenaw Community College - 5 - 5
[DEBUG::] 2395 - Wells College - 1 - 1
[DEBUG::] 2396 - Miami Children's Hospital - 2 - 2
[DEBUG::] 2397 - University of Maryland - University College - 1 - 1
[DEBUG::] 2398 - Lincoln Memorial University - 11 - 11
[DEBUG::] 2399 - Stetson University - 3 - 3
[DEBUG::] 2400 - Gaston College - 3 - 3
[DEBUG::] 2401 - SUNY College of Technology at Alfred - 3 - 3
[DEBUG::] 2402 - John D. Dingell VA Medical Center - 5 - 5
[DEBUG::] 2403 - Portland State University - 9 - 9
[DEBUG::] 2404 - Columbia Basin College - 2 - 2
[DEBUG::] 2405 - Pensacola State College - 10 - 10
[DEBUG::] 2406 - The University of Tennessee - Chattanooga - 14 - 14
[DEBUG::] 2407 - San Diego State University - 22 - 22
[DEBUG::] 2408 - Denver Seminary - 1 - 1
[DEBUG::] 2409 - P

[DEBUG::] 2577 - Boston College - 9 - 9
[DEBUG::] 2578 - Columbia International University - 2 - 2
[DEBUG::] 2579 - Barton County Community College - 1 - 1
[DEBUG::] 2580 - University of the Sacred Heart - 3 - 3
[DEBUG::] 2581 - Anthem College - Phoenix - 6 - 6
[DEBUG::] 2582 - Clarion County Career Center Practical Nursing Program - 1 - 1
[DEBUG::] 2583 - University of Texas Health Science Center Houston - 17 - 17
[DEBUG::] 2584 - Ponce School of Medicine and Health Sciences - 4 - 4
[DEBUG::] 2585 - SUNY Buffalo School of Dental Medicine - 3 - 3
[DEBUG::] 2586 - John A Logan College - 3 - 3
[DEBUG::] 2587 - Amridge University - 1 - 1
[DEBUG::] 2588 - Purdue University - North Central - 4 - 4
[DEBUG::] 2589 - York Technical College - 5 - 5
[DEBUG::] 2590 - San Jacinto College District - 6 - 6
[DEBUG::] 2591 - Southern Illinois University Carbondale - 22 - 22
[DEBUG::] 2592 - Regis University - 9 - 9
[DEBUG::] 2593 - University of South Carolina - Aiken - 5 - 5
[DEBUG::] 2594 - Western 

[DEBUG::] 2768 - Indiana University Southeast - 2 - 2
[DEBUG::] 2769 - Coney Island Hospital - 1 - 1
[DEBUG::] 2770 - Catholic University of America - 11 - 11
[DEBUG::] 2771 - University of Florida - 47 - 47
[DEBUG::] 2772 - Missouri Western State University - 6 - 6
[DEBUG::] 2773 - Marshall B. Ketchum University - 5 - 5
[DEBUG::] 2774 - Rochester College - 1 - 1
[DEBUG::] 2775 - County College of Morris - 2 - 2
[DEBUG::] 2776 - Golden West College - 1 - 1
[DEBUG::] 2777 - University of Portland - 5 - 5
[DEBUG::] 2778 - Mountain Empire Community College - 1 - 1
[DEBUG::] 2779 - Christian Brothers University - 2 - 2
[DEBUG::] 2780 - The Music School of Delaware - 1 - 1
[DEBUG::] 2781 - Tri-County Technical College - 3 - 3
[DEBUG::] 2782 - Germanna Community College - 3 - 3
[DEBUG::] 2783 - McHenry County College - 1 - 1
[DEBUG::] 2784 - Central Virginia Community College - 1 - 1
[DEBUG::] 2785 - Syracuse Central Technical Vocation Center - 1 - 1
[DEBUG::] 2786 - University of West Georg

In [19]:
len(txpool)

13244

In [20]:
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
tx = graph.begin()
[tx.create(e) for e in txpool]
graph.commit(tx)

In [21]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
2863


In [88]:
graph.run("MATCH (n) DETACH DELETE(n)")

IndexError: index out of range